# Consolidate Information
* RFP Fluorescence ($\Delta F / F_0$ and Raw and Photobleaching Corrected) - (N $\times$ T)
* GCaMP **or** GFP Fluorescence ($\Delta F / F_0$ and Raw and Photobleaching Corrected) - (N $\times$ T)
* Neuron Position - (N $\times$ [x,y,z, frame #] $\times$ T)
* Centerline - (T2 $\times$ ~102 $\times$ [x,y])
* Ethogram - (T $\times$ [-1,0,1,2])
* Times in seconds for T and T2

### Worms: *GFP, S1, S2, 5, 6, & 7*

In [ ]:
### Information is saved in .npz format, accessing is simple. Example:
# W4 = np.load('Worm4.npz')
# G = W4['G']

In [ ]:
import numpy as np
import scipy.io as sio

### Data source to use, should be dated folder that is a copy from Andy's server
source = ['20160203_AllData/', '20160216_AllData/']
user = ['nroy','nicholasroy']

Directory = '/Users/' + user[0] + '/CElegans/Data/' + source[-1]

# Put in neuron position across time in cleaner more condensed format
def NPos_clean(x, neuron_count):
    positions = np.zeros((len(x), neuron_count, 3))
    for i_ind, i in enumerate(x):
        if np.count_nonzero(i) != 0: # if no data for whole time point
            for j in range(neuron_count):
                if np.count_nonzero(i[j]) != 0: # if no data for neuron at time point
                    positions[i_ind][j] = i[j][:3].astype(float)
                else:
                    positions[i_ind][j] = None
        else:
            positions[i_ind] = None       
    return positions

## Collect all information from Worm GFP (GFP + RFP)
### 63 Neurons, 2055 Time Points over 376.5 seconds
### Info saved to WormGFP.npz
N = 63, T = 2055 (376.5sec), T2 = 23316 (-0.11 to 382.51sec)

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# and times T ('hasPointsTime')
f1 = sio.loadmat(Directory+'GFP_moving/heatData.mat')

G = np.array(f1['G2']) # 63 x 2055
R = np.array(f1['R2']) # 63 x 2055
ethoTrack = np.array(f1['ethoTrack']) # 2055 x 1
Time = np.array(f1['hasPointsTime']) # 2055 x 1


# Raw fluorescence for both GFP and RFP ('G2' and 'R2')
f2 = sio.loadmat(Directory+'GFP_moving/rawHeatmapData.mat')

G_Raw = np.array(f2['GRaw']) # 63 x 2055
R_Raw = np.array(f2['RRaw']) # 63 x 2055

G_photocorr = None
R_photocorr = None


# Neuron Poistion, xyz coordinates from a single volume when worm was straight ('fiducials')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f3 = sio.loadmat(Directory+'GFP_moving/botFiducials.mat')

NPos = NPos_clean(f3['fiducialPoints'][:,0], len(G)) # 2055 x 63 x 3


# Centerline Position, x coords + y coords + T2 times (each from .csv file)
C_Directory = Directory + 'GFP_moving/Behavior/'

CenterTimes = np.genfromtxt(C_Directory+'centerlineData_gfp_t.csv', delimiter=',').reshape(-1,1) # 23316 x 1
Center_x = np.genfromtxt(C_Directory+'centerlineDatagfp_x.csv', delimiter=',') # 102 x 23316
Center_y = np.genfromtxt(C_Directory+'centerlineData_gfp_y.csv', delimiter=',') # 102 x 23316


# Save all information to a single npz file
np.savez_compressed('WormGFP', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )

## Collect all information from Worm S1 (GCaMP + RFP)
### 82 Neurons, 2771 Time Points over 461 seconds, Worm is immobile
### Info saved to WormS1.npz
N = 82, T = 2771 (461sec), T2 = None

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# and times T ('hasPointsTime')
f1 = sio.loadmat(Directory+'BrainScanner20160121_134707_StationaryWorm/heatData.mat')

G = np.array(f1['G2']) # 82 x 2771
R = np.array(f1['R2']) # 82 x 2771
ethoTrack = None
Time = np.array(f1['hasPointsTime']).T # 2771 x 1

# Raw fluorescence & photbleaching corrected for both GFP & RFP ('gRaw' and 'gPhotoCorr')
G_Raw = np.array(f1['gRaw']) # 82 x 2771
R_Raw = np.array(f1['rRaw']) # 82 x 2771

G_photocorr = np.array(f1['gPhotoCorr']) # 82 x 2771
R_photocorr = np.array(f1['rPhotoCorr']) # 82 x 2771


# Neuron Poistion, xyz coordinates over time ('fiducialPoints')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f2 = sio.loadmat(Directory+'BrainScanner20160121_134707_StationaryWorm/botFiducials.mat')

NPos = NPos_clean(f2['fiducialPoints'][0], len(G)) # 2771 x 82 x 3


CenterTimes = None
Center_x = None
Center_y = None

# Save all information to a single npz file
np.savez_compressed('WormS1', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )

## Collect all information from Worm S2 (GCaMP + RFP)
### 119 Neurons, 2052 Time Points over 342 seconds, Worm is immobile
### Info saved to WormS2.npz
N = 119, T = 2052 (342sec), T2 = None

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# and times T ('hasPointsTime')
f1 = sio.loadmat(Directory+'BrainScanner20160121_140306_StationaryWorm/heatData.mat')

G = np.array(f1['G2'][:,76:2128]) # 119 x 2470
R = np.array(f1['R2'][:,76:2128]) # 119 x 2470
ethoTrack = None
Time = np.array(f1['hasPointsTime'][:,76:2128]).T # 2470 x 1

# Raw fluorescence & photbleaching corrected for both GFP & RFP ('gRaw' and 'gPhotoCorr')
G_Raw = np.array(f1['gRaw'][:,76:2128]) # 119 x 2470
R_Raw = np.array(f1['rRaw'][:,76:2128]) # 119 x 2470

G_photocorr = np.array(f1['gPhotoCorr'][:,76:2128]) # 119 x 2470
R_photocorr = np.array(f1['rPhotoCorr'][:,76:2128]) # 119 x 2470


# Neuron Poistion, xyz coordinates over time ('fiducialPoints')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f2 = sio.loadmat(Directory+'BrainScanner20160121_140306_StationaryWorm/botFiducials.mat')

NPos = NPos_clean(f2['fiducialPoints'][0][76:2128], len(G)) # 2470 x 119 x 3


CenterTimes = None
Center_x = None
Center_y = None

# Save all information to a single npz file
np.savez_compressed('WormS2', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )

## Collect all information from Worm 5 (GCaMP + RFP)
### 98 Neurons, 1440 Time Points over 240 seconds
### Info saved to Worm5.npz
N = 98, T = 1440 (240.33sec), T2 = 15007 (0.0 to 246.17sec)

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# and times T ('hasPointsTime')
f1 = sio.loadmat(Directory+'heatData_worm5.mat')

G = np.array(f1['G2'])[:,2:] # 98 x 1440
R = np.array(f1['R2'])[:,2:] # 98 x 1440
ethoTrack = np.array(f1['ethoTrack'])[1:] # 1440 x 1
Time = np.array(f1['hasPointsTime'])[1:] # 1440 x 1


# Raw fluorescence for both GFP and RFP ('G2' and 'R2')
f2 = sio.loadmat(Directory+'rawHeatmapData_worm5.mat')

G_Raw = np.array(f2['GRaw'])[:,2:] # 98 x 1440
R_Raw = np.array(f2['RRaw'])[:,2:] # 98 x 1440

G_photocorr = None
R_photocorr = None


# Neuron Poistion, xyz coordinates from a single volume when worm was straight ('straightPoints')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f3 = sio.loadmat(Directory+'botFiducials_worm5.mat')

NPos = NPos_clean(f3['fiducialPoints'][0,2:], len(G)) # 1440 x 98 x 3


# Centerline Position, x coords + y coords + T2 times (each from .csv file)
C_Directory = Directory + 'Behavior_worm5/'

CenterTimes = np.genfromtxt(C_Directory+'centerlineData_worm4_time.csv', delimiter=',').reshape(-1,1) # 15007 x 1
Center_x = np.genfromtxt(C_Directory+'centerlineData_worm4_x.csv', delimiter=',') # 102 x 15007
Center_y = np.genfromtxt(C_Directory+'centerlineData_worm4_y.csv', delimiter=',') # 102 x 15007


# Save all information to a single npz file
np.savez_compressed('Worm5', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )

## Collect all information from Worm 6 (GCaMP + RFP)
### 95 Neurons, 1904 Time Points over 317 seconds
### Info saved to Worm6.npz
N = 95, T = 1904 (317.33sec), T2 = 17570 (17.99 to 347.29sec)

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# times T ('hasPointsTime')
f1 = sio.loadmat(Directory+'heatData_worm6.mat')

G = np.array(f1['G2'])[:,2:] # 95 x 1904
R = np.array(f1['R2'])[:,2:] # 95 x 1904
ethoTrack = np.array(f1['ethoTrack'])[1:] # 1904 x 1
Time = np.array(f1['hasPointsTime'])[1:] # 1904 x 1


# Raw fluorescence for both GFP and RFP ('G2' and 'R2')
f2 = sio.loadmat(Directory+'rawHeatmapData_worm6.mat')

G_Raw = np.array(f2['GRaw'])[:,2:] # 95 x 1904
R_Raw = np.array(f2['RRaw'])[:,2:] # 95 x 1904

G_photocorr = None
R_photocorr = None


# Neuron Poistion, xyz coordinates over time ('fiducialPoints')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f3 = sio.loadmat(Directory+'botFiducials_worm6.mat')

NPos = NPos_clean(f3['fiducialPoints'][0,2:], len(G)) # 1904 x 95 x 3


# Centerline Position, x coords + y coords + T2 times (each from .csv file)
C_Directory = Directory + 'Behavior_worm6/'

CenterTimes = np.genfromtxt(C_Directory+'centerlineData_worm1_time.csv', delimiter=',').reshape(-1,1) # 17570 x 1
Center_x = np.genfromtxt(C_Directory+'centerlineData_worm1_x.csv', delimiter=',') # 102 x 17570
Center_y = np.genfromtxt(C_Directory+'centerlineData_worm1_y.csv', delimiter=',') # 102 x 17570


# Save all information to a single npz file
np.savez_compressed('Worm6', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )

## Collect all information from Worm 7 (GCaMP + RFP)
### 156 Neurons, 2848 Time Points over 474 seconds
### Info saved to Worm7.npz
N = 156, T = 2848 (474.66sec), T2 = 30394 (-4.32 to 511.22sec)

In [ ]:
# Delta_F / F_0 for both GFP and RFP ('G2' and 'R2'), ethogram ('ethoTrack', no 0 values)
# times T ('hasPointsTime'), 
f1 = sio.loadmat(Directory+'BrainScanner20141014_143957_longmovingworm/heatData.mat')

G = np.array(f1['G2'])[:,1:] # 156 x 2848
R = np.array(f1['R2'])[:,1:] # 156 x 2848
ethoTrack = np.array(f1['ethoTrack'])[1:] # 2848 x 1
Time = np.array(f1['hasPointsTime'])[1:] # 2848 x 1

# Raw fluorescence & photbleaching corrected for both GFP & RFP ('gRaw' and 'gPhotoCorr')
G_Raw = np.array(f1['gRaw'])[:,1:] # 156 x 2848
R_Raw = np.array(f1['rRaw'])[:,1:] # 156 x 2848

G_photocorr = np.array(f1['gPhotoCorr'])[:,1:] # 156 x 2848
R_photocorr = np.array(f1['rPhotoCorr'])[:,1:] # 156 x 2848


# Neuron Poistion, xyz coordinates over time ('fiducialPoints')
# Note that x&y are in pixels, z is in volts with 1V ~ 30 pixels
f2 = sio.loadmat(Directory+'BrainScanner20141014_143957_longmovingworm/botFiducials.mat')

NPos = NPos_clean(f2['fiducialPoints'][0,1:], len(G)) # 2848 x 156 x 3


# Centerline Position, x coords + y coords
C_Directory = Directory + 'BrainScanner20141014_143957_longmovingworm/BehaviorAnalysis/'

f3 = sio.loadmat(C_Directory+'centerline.mat')

Center_x = np.array(f3['centerline'])[:,0] # 100 x 30394
Center_y = np.array(f3['centerline'])[:,1] # 100 x 30394


# Centerline Times
f4 = sio.loadmat(C_Directory+'centerlineTime.mat')

CenterTimes = np.array(f4['bfTime']) # 30394 x 1


# Save all information to a single npz file
np.savez_compressed('Worm7', 
        G=G, R=R, ethoTrack=ethoTrack, Time=Time, 
        G_Raw=G_Raw, R_Raw=R_Raw, G_photocorr=G_photocorr, R_photocorr=R_photocorr,
        NPos=NPos,
        CenterTimes=CenterTimes, Center_x=Center_x, Center_y=Center_y
                   )